In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imread
# from skimage import data_dir
from skimage.transform import radon, iradon, rescale
image = imread("./Shepp_logan.jpg", as_gray=True)
image = rescale(image, scale=0.4)
plt.figure(figsize=(8, 8.5))
plt.subplot(221)
plt.title("Original")
plt.imshow(image, cmap=plt.cm.Greys_r)
plt.subplot(222)
projections = radon(image, theta=[0, 45, 90])
plt.plot(projections)
plt.title("Projections at\n0, 45 and 90 degrees")
plt.xlabel("Projection axis")
plt.ylabel("Intensity")
projections = radon(image)
plt.subplot(223)
plt.title("Radon transform\n(Sinogram)")
plt.xlabel("Projection axis")
plt.ylabel("Intensity")
plt.imshow(projections)
reconstruction = iradon(projections)
plt.subplot(224)
plt.title("Reconstruction\nfrom sinogram")
plt.imshow(reconstruction, cmap=plt.cm.Greys_r)
plt.subplots_adjust(hspace=0.4, wspace=0.5)
plt.show()

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from helpers.image_exec import ImageExec
import math

image = list()

pxs = 1000

for i in range(pxs + 1):
  row = list()
  for j in range(pxs + 1):
    row.append((0,0,0))
  image.append(row)

image = np.asarray(image)
# print(image)



from ipywidgets import interact


im = ImageExec(image)

# im.draw_bresenham_line((0,0),(99,95), (255,255,255))

# print(im.calc_bresenham_line((0,0),(0,99)))
sr = int(pxs / 2)
r = int(sr * math.sqrt(2));
# im.set_pixel(sr, r, color=(255,255,255))

def f(x):
    im.clear_image()
    alpha = math.radians(x)
    fi = math.radians(10)
    n = 15

    E_x = int(r * math.cos(alpha)) + sr
    E_y = int(r * math.sin(alpha)) + sr

    for i in range(n):
      

      D_x = int(r * math.cos(alpha + math.pi - fi / 2 + i * (fi / (n - 1)))) + sr
      D_y = int(r * math.sin(alpha + math.pi - fi / 2 + i * (fi / (n - 1)))) + sr
      im.draw_bresenham_line((E_x,E_y),(D_x,D_y), (255,255,255))

    # im.set_pixel(E_x, E_y, (255,0,0));

    # print(im.calc_bresenham_line((E_x,E_y),(D_x,D_y)))

    plt.imshow(im.image)
    return x

interact(f, x = (0, 360))



interactive(children=(IntSlider(value=180, description='x', max=360), Output()), _dom_classes=('widget-interac…

<function __main__.f(x)>